In [1]:
AGENT_NAME = '20 bin PPO 500 results\default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = r'20 bin PPO 500 results' + '/'
EPS = 0.05
ATK_NAME = f'untageted_AC_5%_eps'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion import AutoConjugateGradient as ACG
#from KBMproject.mybb import BrendelBethgeAttack as BBA
from art.estimators.classification import PyTorchClassifier as classifier
from art.utils import to_categorical

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

import KBMproject.utilities as utils

import json
from tqdm import tqdm

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
try: #try to load CityLearn schema
    schema = DataSet.get_schema(DATASET_NAME)
except: #load saved schema otherwise
    with open(DATASET_NAME, 'r') as file:
        schema = json.load(file)

Define RL agent

In [5]:
agent = PPO.load(path=f"{AGENT_NAME}",
                 print_system_info=True)
print('Model loaded from storage')

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.0
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

Model loaded from storage


In [6]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [7]:
cols = env.observation_names

In [8]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features

In [9]:
agent_classifier = classifier(
        model=utils.extract_actor(agent),
        loss=utils.CWLoss(), 
        nb_classes=agent.action_space[0].n,
        input_shape=agent.observation_space.shape,
        device_type='gpu',
        clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max()) #min and max values of each feature, brendle bethge attack only supports floats values and not array
        )

In [10]:
init = 50
iter = int(1000/init)
import logging
logging.basicConfig(level=logging.WARN)
kwargs = {"loss_type": 'difference_logits_ratio', 
          "eps": EPS, 
          "eps_step": 2*EPS, 
          "batch_size": 1, 
          "nb_random_init": init, 
          "max_iter": iter, 
          "norm": "inf", 
          "verbose": False}
attack = ACG(estimator=agent_classifier, **kwargs,)

In [11]:
%%time
kpis, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, 
                                                                        env, 
                                                                        attack,
                                                                        time_steps=None,
                                                                        mask=observation_masks)

100%|██████████| 8759/8759 [52:21<00:00,  2.79it/s, ASR=0.957]


CPU times: total: 8min 27s
Wall time: 52min 22s


In [12]:
with open(SAVE_DIR+f'{ATK_NAME} parameters.json', 'w') as f:
    json.dump(kwargs, f)

In [13]:
display(kpis)

cost_function
annual_peak_average                      1.264611
carbon_emissions_total                   0.909828
cost_total                               0.828607
daily_one_minus_load_factor_average      1.160861
daily_peak_average                       0.998033
electricity_consumption_total            0.918737
monthly_one_minus_load_factor_average    1.003955
ramping_average                          1.326874
zero_net_energy                          1.110196
Name: District, dtype: float64

In [14]:
kpi = kpis

In [15]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

20 bin PPO 500 results/KPIs.csv updated


In [17]:
df_obs = pd.DataFrame(obs)
df_obs.columns = cols
#df_obs['a'] = actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+' a-obs.csv')

In [18]:
df_obs = pd.DataFrame(adv_obs)
df_obs.columns = cols
#df_obs['a'] = adv_actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+' adv a-obs.csv')

In [19]:
asr_savename = SAVE_DIR+'ASRs.csv'
try:
    df_asrs = pd.read_csv(asr_savename)
    df_asrs[ATK_NAME] = asr
    df_asrs.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    asr = pd.Series([asr])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print(f'{asr_savename} created')

20 bin PPO 500 results/ASRs.csv created
